## 텐서플로우를 활용하여 신경망 구현하기 - 데이터 전 처리

* 마케터로서 광고 비용에 따른 수익률을 신경망을 통해서 예측하고자 합니다.
* FB,TV,Newspaper에 따른 Sales 예측하기

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
np.random.seed(100)
tf.random.set_seed(100)

In [5]:
# 데이터를 DataFrame 형태로 불러 옵니다.
df = pd.read_csv("Advertising.csv")

In [6]:
print('원본 데이터 샘플 :')
print(df.head(),'\n')

원본 데이터 샘플 :
   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 



In [7]:
# 의미없는 변수는 삭제합니다.
df = df.drop(columns=['Unnamed: 0'])

"""
1. Sales 변수는 label 데이터로 Y에 저장하고 나머진 X에 저장합니다.
"""
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

In [8]:
"""
2. 학습용 데이터를 tf.data.Dataset 형태로 변환합니다.
   from_tensor_slices 함수를 사용하여 변환하고 batch를 수행하게 합니다.
"""
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)  # batch는 학습할 때만 사용한다는 거!

In [9]:
# 하나의 batch를 뽑아서 feature와 label로 분리합니다.
[(train_features_batch, label_batch)] = train_ds.take(1)

In [10]:
# batch 데이터를 출력합니다.
# bathch size 5니까, 5개 출력
print('\nFB, TV, Newspaper batch 데이터:\n',train_features_batch)
print('Sales batch 데이터:',label_batch)


FB, TV, Newspaper batch 데이터:
 tf.Tensor(
[[296.4  36.3 100.9]
 [228.   37.7  32. ]
 [  5.4  29.9   9.4]
 [ 57.5  32.8  23.5]
 [240.1   7.3   8.7]], shape=(5, 3), dtype=float64)
Sales batch 데이터: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


## 텐서플로우를 활용하여 신경망 구현하기 - 모델 구현

* 텐서플로우와 케라스(Keras)를 활용하여 신경망 모델을 구현  
* |케라스는 텐서플로우 내의 신경망 모델 설계와 훈련을 위한 API 입니다. 케라스는 연속적으로(Sequential) 레이어(Layer)들을 쌓아가며 모델을 생성하고, 사이킷런과 같이 한 줄의 코드로 간단하게 학습 방법 설정, 학습, 평가를 진행할 수 있습니다.

In [11]:
"""
1. tf.keras.models.Sequential()를 활용하여 신경망 모델을 생성합니다.
   자유롭게 layers를 쌓고 마지막 layers는 노드 수를 1개로 설정합니다.
"""
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(3,)),    # FB, TV, Newspaper 3개
    tf.keras.layers.Dense(1)
    ])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                40        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
None


## 텐서플로우를 활용하여 신경망 구현하기 - 모델 학습

* 텐서플로우와 케라스(Keras)를 활용하여 신경망 모델을 학습  
 
* 텐서플로우와 케라스를 이용해 신경망 모델을 학습하기 위한 함수/메서드  

* complie() 메서드는 모델을 어떻게 학습할 지에 대해서 설정합니다.  
loss는 회귀에서는 일반적으로 MSE인 ‘mean_squared_error’, 분류에서는 ‘sparse_categorical_crossentropy’ 를 주로 사용합니다.

In [13]:
"""
1. 학습용 데이터를 바탕으로 모델의 학습을 수행합니다.
    
step1. compile 메서드를 사용하여 최적화 모델 설정합니다.
       loss는 mean_squared_error, optimizer는 adam으로 설정합니다.
       
step2. fit 메서드를 사용하여 Dataset으로 변환된 학습용 데이터를 학습합니다.
       epochs는 100으로 설정합니다.
"""

model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs=100, verbose=2) 
# verbose 인자는 학습 시, 화면에 출력되는 형태를 설정합니다. (0: 표기 없음, 1: 진행 바, 2: 에포크당 한 줄 출력)

Epoch 1/100
28/28 - 1s - loss: 1746.2517
Epoch 2/100
28/28 - 0s - loss: 323.6100
Epoch 3/100
28/28 - 0s - loss: 158.4601
Epoch 4/100
28/28 - 0s - loss: 109.7061
Epoch 5/100
28/28 - 0s - loss: 75.6144
Epoch 6/100
28/28 - 0s - loss: 52.3663
Epoch 7/100
28/28 - 0s - loss: 36.6032
Epoch 8/100
28/28 - 0s - loss: 26.3099
Epoch 9/100
28/28 - 0s - loss: 20.1578
Epoch 10/100
28/28 - 0s - loss: 15.9902
Epoch 11/100
28/28 - 0s - loss: 13.1412
Epoch 12/100
28/28 - 0s - loss: 11.0550
Epoch 13/100
28/28 - 0s - loss: 9.5409
Epoch 14/100
28/28 - 0s - loss: 8.2736
Epoch 15/100
28/28 - 0s - loss: 7.3852
Epoch 16/100
28/28 - 0s - loss: 6.6189
Epoch 17/100
28/28 - 0s - loss: 5.9925
Epoch 18/100
28/28 - 0s - loss: 5.6492
Epoch 19/100
28/28 - 0s - loss: 5.2800
Epoch 20/100
28/28 - 0s - loss: 4.9974
Epoch 21/100
28/28 - 0s - loss: 4.8657
Epoch 22/100
28/28 - 0s - loss: 4.6492
Epoch 23/100
28/28 - 0s - loss: 4.5337
Epoch 24/100
28/28 - 0s - loss: 4.4366
Epoch 25/100
28/28 - 0s - loss: 4.4334
Epoch 26/100
28/2

## 텐서플로우를 활용하여 신경망 구현하기 - 모델 평가 및 예측
* evaluate() 메서드는 학습된 모델을 바탕으로 입력한 feature 데이터 X와 label Y의 loss 값과 metrics 값을 출력합니다.   
model.evaluate(X, Y)


* 이번 실습에서는 metrics 를 compile에서 설정하지 않았지만, 분류에서는 일반적으로 accuracy를 사용하여 evaluate 사용 시, 2개의 아웃풋을 리턴합니다

In [14]:
"""
1. evaluate 메서드를 사용하여 테스트용 데이터의 loss 값을 계산합니다.
"""
loss = model.evaluate(test_X, test_Y, verbose=0)

"""
2. predict 메서드를 사용하여 테스트용 데이터의 예측값을 계산합니다.
"""
predictions = model.predict(test_X)

# 결과를 출력합니다.
print("테스트 데이터의 Loss 값: ", loss)
for i in range(5):
    print("%d 번째 테스트 데이터의 실제값: %f" % (i, test_Y.iloc[i]))
    print("%d 번째 테스트 데이터의 예측값: %f" % (i, predictions[i][0]))

테스트 데이터의 Loss 값:  2.534884452819824
0 번째 테스트 데이터의 실제값: 6.600000
0 번째 테스트 데이터의 예측값: 10.425423
1 번째 테스트 데이터의 실제값: 20.700000
1 번째 테스트 데이터의 예측값: 19.977482
2 번째 테스트 데이터의 실제값: 17.200000
2 번째 테스트 데이터의 예측값: 17.103910
3 번째 테스트 데이터의 실제값: 19.400000
3 번째 테스트 데이터의 예측값: 19.590137
4 번째 테스트 데이터의 실제값: 21.800000
4 번째 테스트 데이터의 예측값: 21.168655


### 결과 해석: 

1.  학습용 데이터의 loss값과 테스트 데이터의 loss값을 비교한다.
>> 학습할 때 loss 값 보다, 테스트 데이터의 loss값이 더 적다는 것을 확인할 수 있다.
2. 테스트 데이터의 실제값과 예측값을 비교한다. (for 반복문 활용)

4 번째 테스트 데이터의 실제값: 21.800000  
4 번째 테스트 데이터의 예측값: 20.184616  